In [ ]:
#Create an Enterprise Geodatabse and import the XML pieces to test the model development in Khub DEV
#When re-iterating a deleted database, you can keep the existing sde connecitions

from arcpy import CreateEnterpriseGeodatabase_management, CreateDatabaseConnection_management, CreateDatabaseUser_management
from getpass import getpass

sapass = getpass('sysadmin password')
sdepass = getpass('sde password')
rhpass = getpass('RH password')
DBName = "KHub5"
CreateEnterpriseGeodatabase_management("SQL_Server", r"khdbtransdev\khub_dev", DBName, "DATABASE_AUTH", "geo_admin", sapass, "SDE_SCHEMA", "sde", sdepass, "", "C:/gisdata/keycodes/License10.6/sysgen/keycodes")

rootpath = r"C:/gisdata/DBConnections/Khub/KHDev"


In [ ]:
rootpath = r"C:/gisdata/DBConnections/Khub/KHDev"

CreateDatabaseConnection_management(rootpath, "sde@"+DBName, "SQL_Server", r"khdbtransdev\khub_dev", "DATABASE_AUTH", "sde", sdepass, "SAVE_USERNAME", DBName)
CreateDatabaseConnection_management(rootpath, "sa@"+DBName, "SQL_Server", r"khdbtransdev\khub_dev", "DATABASE_AUTH", "geo_admin", sapass, "SAVE_USERNAME", DBName)
CreateDatabaseUser_management(rootpath+"/sa@"+DBName+".sde",  "DATABASE_USER", "rh", rhpass)
CreateDatabaseConnection_management(rootpath, "rh@"+DBName, "SQL_Server", r"khdbtransdev\khub_dev", "DATABASE_AUTH", "rh", rhpass, "SAVE_USERNAME", DBName)


In [8]:
from arcpy import env
env.Mtolerance = '0.00000062136995'
env.outputMFlag = 'Enabled'
env.OutputZFlag = 'Enabled'
env.XYTolerance = '0.001'


In [ ]:
from arcpy import management
DBName = "KHub5"
rootpath = r"C:/gisdata/DBConnections/Khub/KHDev"
proALRSdb = rootpath+r'\rh@'+DBName+'.sde'
XML_Model = r"C:\temp\validate9.xml"

management.ImportXMLWorkspaceDocument(proALRSdb, XML_Model, "SCHEMA_ONLY", "NETWORK_GEOMETRY")

In [ ]:
#this requires an oracle client to use SHARED.URBAN_BOUNDARIES at this time AR55 has no oracle client
#I ran these commands in Pro on my laptop - kyle g
#SHARED.URBAN_BOUNDARIES has duplicate rows that we assume to be distinct with 'pop_year = 2010'
#Then I entered the coded values into the model and changed domains to duplicate split policies

from arcpy import conversion
conversion.TableToTable("SHARED.URBAN_BOUNDARIES", r"C:\temp\UrbanBoundaryDomain.gdb", "UAB_Table", "POP_YEAR = 2010")
management.TableToDomain(r"C:\temp\UrbanBoundaryDomain.gdb\UAB_Table", "URBANCITYNO", "URBANCITYNAME", r"C:\temp\UrbanBoundaryDomain.gdb", "dUrbanJurisdiction", "Urban Boundary Jurisdiction ", "APPEND")

In [14]:
from arcpy import management, conversion

DBName = "KHub5"

rootpath = r"C:/gisdata/DBConnections/Khub/KHDev"
sourceALRS = r'C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde'
proALRSdb = rootpath+r'\rh@'+DBName+'.sde'

projVert = "PROJCS['NAD_1983_Kansas_LCC_ftUS',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Lambert_Conformal_Conic'],PARAMETER['False_Easting',1312333.3333],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-98.25],PARAMETER['Standard_Parallel_1',37.5],PARAMETER['Standard_Parallel_2',39.5],PARAMETER['Latitude_Of_Origin',36.0],UNIT['Foot_US',0.3048006096012192]],VERTCS['NAVD88_height_(ftUS)',VDATUM['North_American_Vertical_Datum_1988'],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Foot_US',0.3048006096012192]];-120729000 -94795000 3048.00609601219;-100000 10000;-100000 10000;3.28083333333333E-03;0.001;0.001;IsHighPrecision"

from arcpy import env
env.Mtolerance = '0.00000062136995'
env.outputMFlag = 'Enabled'
env.OutputZFlag = 'Enabled'
env.XYTolerance = '0.001'

management.CreateFeatureDataset(rootpath+r'\rh@'+DBName+'.sde', "ALRS", projVert)




<Result 'C:\\gisdata\\DBConnections\\Khub\\KhDev\\rh@KHub5.sde\\KHub5.RH.ALRS'>

In [15]:
from arcpy import env
env.Mtolerance = '0.00000062136995'
env.outputMFlag = 'Enabled'
env.OutputZFlag = 'Enabled'
env.XYTolerance = '0.001'


conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.Redline", proALRSdb+r"\\"+DBName+".RH.ALRS", "Redline", '', '', "NETWORK_GEOMETRY")
conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.Calibration_Point", proALRSdb+r"\\"+DBName+".RH.ALRS", "Calibration_Point", '', '', "NETWORK_GEOMETRY")
conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.Centerline", proALRSdb+r"\\"+DBName+".RH.ALRS", "Centerline", '', '', "NETWORK_GEOMETRY")
conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.LRS_County", proALRSdb+r"\\"+DBName+".RH.ALRS", "LRS_County", '', '', "NETWORK_GEOMETRY")
conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.LRS_State", proALRSdb+r"\\"+DBName+".RH.ALRS", "LRS_State", '', '', "NETWORK_GEOMETRY")
conversion.TableToGeodatabase(sourceALRS+r"\KHUB.RH.Centerline_Sequence", proALRSdb)
conversion.TableToGeodatabase(sourceALRS+r"\KHUB.RH.Lrs_Edit_Log", proALRSdb)
conversion.TableToTable(sourceALRS+r"\KHUB.RH.LRS_EVENT_BEHAVIOR", proALRSdb, "LRS_EVENT_BEHAVIOR", "1=1")    
conversion.TableToTable(sourceALRS+r"\KHUB.RH.LRS_METADATA", proALRSdb, "LRS_METADATA", "1=1")  
conversion.TableToTable(sourceALRS+r"\KHUB.RH.Lrs_Locks", proALRSdb, "Lrs_Locks", "1=0") 



<Result 'C:\\gisdata\\DBConnections\\Khub\\KhDev\\rh@KHub5.sde\\KHub5.RH.Lrs_Locks'>

In [16]:
from arcpy import locref
DBName = "KHub5"

locref.ModifyLRSNetwork(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH.LRS_County", "RouteId", None, "LRSFromDate", "LRSToDate", "DO_NOT_DERIVE", '', "DO_NOT_INCLUDE", None, None, None, "MULTI_FIELD_ROUTE_ID", "County;Prefix;RouteNum;Suffix;InvDir;UniqueId")

<Result 'C:\\gisdata\\DBConnections\\Khub\\KHDev\\rh@KHub5.sde\\\\KHub5.RH.ALRS\\KHub5.RH.LRS_County'>

In [17]:
locref.ModifyLRSNetwork(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH.LRS_State", "RouteId", None, "LRSFromDate", "LRSToDate", "DO_NOT_DERIVE", '', "DO_NOT_INCLUDE", None, None, None, "MULTI_FIELD_ROUTE_ID", "Prefix;RouteNum;Suffix;InvDir;UniqueId")

<Result 'C:\\gisdata\\DBConnections\\Khub\\KHDev\\rh@KHub5.sde\\\\KHub5.RH.ALRS\\KHub5.RH.LRS_State'>

In [18]:
EventList = ['ev_AADT',
            'ev_AADT2012',
            'ev_AADT2013',
            'ev_AADT2014',
            'ev_AADT2015',
            'ev_AADT2016',
            'ev_AADTDirFactor',
            'ev_AADTExpansionFactor',
            'ev_AADTKFactor',
            'ev_AccessControl',
            'ev_AccidentAnalysisDate',
            'ev_AdministrativeOwner',
            'ev_AdvisorySpeed',
            'ev_AuxiliaryLane',
            'ev_CapacityHCM',
            'ev_CityJurisdiction',
            'ev_CollectorSection',
            'ev_CongressionalDistrict',
            'ev_County',
            'ev_CountyOnState',
            'ev_CurbType',
            'ev_DesignAccessControl',
            'ev_DesignStandShoulderWt',
            'ev_EPFS',
            'ev_EPFS2012',
            'ev_EPFS2013',
            'ev_EPFS2014',
            'ev_EPFS2015',
            'ev_EPFS2016',  #need to add additional EPFS Fields eventually?
            'ev_FieldInventory',
            'ev_FreightNetwork',
            'ev_FunctionalClass',
            'ev_GradientPercent',
            'ev_HorizontalCurve',
            'ev_HorizontalCurveFT',
            'ev_HPMSPavementCondition',
            'ev_HPMSReporting',
            'ev_HPMSSample',
            'ev_IndianNation',  #Check Metadata
            'ev_KDOTMaintenance',
            'ev_KDOTProjects',
            'ev_KDOTRouteClass',
            'ev_LegacyRoute', #need to check metadata
            'ev_MaintenanceResp',
            'ev_MajorGrading',
            'ev_ManagedLanes',
            'ev_Median',
            'ev_MedianBarrierType',
            'ev_MedianCrossoverType',
            'ev_MPOJurisdiction',
            'ev_MuniROW',
            'ev_NG911RouteStatus',
            'ev_NG911StreetName',
            'ev_NHSFederalAid',
            'ev_NonStateRouteStatus',
            'ev_NumberLanesAggregated',
            'ev_NumberThroughLanes',
            'ev_OnStreetParkingLeft',
            'ev_OnStreetParkingRight',
            'ev_OutdoorAdvertising',
            'ev_PassingLeft',
            'ev_PassingRight',
            'ev_PavementActivity',
            'ev_PavementConditionPSR',
            'ev_PavementCondSummary',
            'ev_PavementManagementSys',
            'ev_PriorityOptPvmtEquiv',
            'ev_ProjectTracking',
            'ev_RightOfWayLeft',
            'ev_RightOfWayMedian',
            'ev_RightOfWayRight',
            'ev_RockWedgeLeft',
            'ev_RockWedgeMedian',
            'ev_RockWedgeRight',
            'ev_RouteDesignation',
            'ev_RumbleStripCenterline',
            'ev_RumbleStripLeft',
            'ev_RumbleStripMedian',
            'ev_RumbleStripRight',
            'ev_SafetyAnalyst',
            'ev_ShoulderMaterialLeft',
            'ev_ShoulderMaterialMed',
            'ev_ShoulderMaterialRight',
            'ev_ShoulderWidthLeft',
            'ev_ShoulderWidthMedian',
            'ev_ShoulderWidthRight',
            'ev_SideSlopeLeft',
            'ev_SideSlopeMedian',
            'ev_SideSlopeRight',
            'ev_SNICE',
            'ev_SpeedLimit',
            'ev_STRAHNET',
            'ev_StripMapMajorGrading',
            'ev_StructureSegment',
            'ev_SupplementalGrading',
            'ev_SurfaceTypeNSRural',
            'ev_SurfaceWidthPaved',
            'ev_TerrainType',
            'ev_ThroughLaneWidth',
            'ev_TrafficSequence',
            'ev_TravelWayOperation',
            'ev_TruckRoute',
            'ev_TurnLeft',
            'ev_TurnRight',
            'ev_UrbanJurisdiction',
            'ev_WideningObstacle',
            'ev_WorkZone'
    ]


PointEvents = [#'ev_Accidents',  #crashes on accident layer
            'ev_AccessPermit',
            'ev_CrestVertSightDistNP',
            'ev_CrestVertSightDistP',
            'ev_Structure',
            'ev_Culverts',
            'ev_RailroadCrossing',
            'ev_ReferencePost',
    
]


MissingEvents = [
    'ev_Accidents',
    'ev_CountyOnCounty',
    'ev_PavementManagementSystem',
    'ev_ShoulderMaterialMedian',
    'ev_SurfaceTypeNonStateRural'    
]

In [21]:
#when the ALRS NEtworks are copied from the source database, 
#the domains are copied with _1 appended to the domain name.
#the domains shouold be remapped to the modeled domains.
#the example here is for LRS_State, this also needs to be done for LRS County

DBName = "KHub5"

rootpath = r"C:/gisdata/DBConnections/Khub/KHDev"
proALRSdb = rootpath+r'\rh@'+DBName+'.sde'
pathsub = proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+"."
management.AssignDomainToField(pathsub+"RH.LRS_State", "Prefix", "dPrefix", None)
management.AssignDomainToField(pathsub+"RH.LRS_State", "Suffix", "dSuffix", None)
management.AssignDomainToField(pathsub+"RH.LRS_State", "InvDir", "dInvDir", None)

management.AssignDomainToField(pathsub+"RH.LRS_County", "County", "dCounty", None)
management.AssignDomainToField(pathsub+"RH.LRS_County", "Prefix", "dPrefix", None)
management.AssignDomainToField(pathsub+"RH.LRS_County", "Suffix", "dSuffix", None)
management.AssignDomainToField(pathsub+"RH.LRS_County", "InvDir", "dInvDir", None)

management.AssignDomainToField(pathsub+"RH.Redline", "ActivityType", "dActivityType", None)


<Result 'C:/gisdata/DBConnections/Khub/KHDev\\rh@KHub5.sde\\\\KHub5.RH.ALRS\\KHub5.RH.Redline'>

In [ ]:
DBName = "KHub5"
proALRSdb = r'C:\gisdata\DBConnections\Khub\KHDev\rh@'+DBName+'.sde'
from arcpy import ExecuteError, locref
import sys

AllEvents = EventList+PointEvents+MissingEvents

for event in AllEvents:
    eventname = event[3:]
    print(eventname)
    try:
        locref.RemoveLRSEntity(proALRSdb, "EVENT", DBName+".RH."+eventname)
    except ExecuteError:
        e = sys.exc_info()[1]
        print(e.args[0])

In [31]:
from arcpy import env, locref, conversion, Exists, management, ExecuteError

#add getcount statements for custom tranformantions and overall recordkeeping

DBName = "KHub5"
sourceALRS = r'C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde'
proALRSdb = r'C:\gisdata\DBConnections\Khub\KHDev\rh@'+DBName+'.sde'

env.Mtolerance = '0.00000062136995'
env.outputMFlag = 'Enabled'
env.outputZFlag = 'Enabled'
env.XYTolerance = '0.001 feet'

renamestring = "KHub20_RH_"

for event in EventList:
    importfeature = event
    ALRSevent = proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH."+importfeature
    ModelEvent = proALRSdb+r"\\"+DBName+".RH."+renamestring+event
    #print(event+", using model "+ModelEvent)
   
    if Exists(ALRSevent):
        eventcount = management.GetCount(ALRSevent)

        print(importfeature + " Exists in ALRS Dataset with "+str(eventcount)+" features ")
        if int(eventcount.getOutput(0)) > 0:
            try:
                locref.CreateLRSEventFromExistingDataset(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH.LRS_County", ALRSevent, "EventID", "RouteID", "LRSFromDate", "LRSToDate", "LocError", "FromMeasure", "ToMeasure", "NO_SPANS_ROUTES", None, "NO_STORE_ROUTE_NAME", None, None)
                print("registered "+str(ALRSevent))
            except ExecuteError:
                e = sys.exc_info()[1]
                print(e.args[0])
        else:
            print("no features in "+str(ALRSevent)+", not registered ")
            
    elif Exists(ModelEvent):
        #print(importfeature + " Using Model")
        conversion.FeatureClassToFeatureClass(ModelEvent, proALRSdb+r"\\"+DBName+".RH.ALRS", importfeature, '1=0', '',"NETWORK_GEOMETRY")
        management.AlterField(ALRSevent, "RouteID", "RouteID", "RouteID", "TEXT", "13", "NULLABLE")
        management.DeleteFeatures(ModelEvent)
        try:
            management.Append(sourceALRS+r"\KHUB.RH."+event, ALRSevent, "NO_TEST", '', '', 'Len(RouteID)<14')
            #locref.CreateLRSEventFromExistingDataset(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH.LRS_County", ALRSevent, "EventID", "RouteID", "LRSFromDate", "LRSToDate", "LocError", "FromMeasure", "ToMeasure", "NO_SPANS_ROUTES", None, "NO_STORE_ROUTE_NAME", None, None)
            eventcount = management.GetCount(ALRSevent)
            sourcecount = management.GetCount(sourceALRS+r"\KHUB.RH."+event)
            print(str(eventcount)+" of "+ str(sourcecount)+" Prod features Appended to model event for "+event)
        except:
            print("cant append prod data for "+event)
    else:
        
        conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH."+event, proALRSdb+r"\\"+DBName+".RH.ALRS", event, '1=0', '',"NETWORK_GEOMETRY")
        management.AlterField(proALRSdb+r'\\'+DBName+'.RH.ALRS\\'+ event, "RouteID", "RouteID", "RouteID", "TEXT", "13", "NULLABLE")
        management.Append(sourceALRS+r"\KHUB.RH."+event, proALRSdb+r'\\'+DBName+'.RH.ALRS\\'+ event, "NO_TEST", '', '', 'Len(RouteID)<14')
        eventcount = management.GetCount(ALRSevent)
        sourcecount = management.GetCount(sourceALRS+r"\KHUB.RH."+event)
        print(str(eventcount)+" of "+ str(sourcecount)+" features from prod added to "+event + " not from model")
        #print('creating event')
        #locref.CreateLRSEventFromExistingDataset(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH.LRS_County", ALRSevent, "EventID", "RouteID", "LRSFromDate", "LRSToDate", "LocError", "FromMeasure", "ToMeasure", "NO_SPANS_ROUTES", None, "NO_STORE_ROUTE_NAME", None, None)
        
for event in PointEvents:
    importfeature = event
    ALRSevent = proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH."+importfeature
    ModelEvent = proALRSdb+r"\\"+DBName+".RH."+renamestring+event
    
    #print(event)
    
    if Exists(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH."+event):
        eventcount = management.GetCount(ALRSevent)
        print(importfeature + " Exists in ALRS Dataset with "+str(eventcount)+" features ")
        if int(eventcount.getOutput(0)) > 0:
            try:
                locref.CreateLRSEventFromExistingDataset(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH.LRS_County", ALRSevent, "EventID", "RouteID", "LRSFromDate", "LRSToDate", "LocError", "Measure")
                print("registered "+str(ALRSevent))
            except ExecuteError:
                e = sys.exc_info()[1]
                print(e.args[0])
        else:
            print("no features in "+str(ALRSevent)+", not registered ")
    elif Exists(ModelEvent):
        conversion.FeatureClassToFeatureClass(ModelEvent, proALRSdb+r"\\"+DBName+".RH.ALRS", importfeature, '1=0', '',"NETWORK_GEOMETRY")
        management.AlterField(ALRSevent, "RouteID", "RouteID", "RouteID", "TEXT", "13", "NULLABLE")
        management.DeleteFeatures(ModelEvent)
        try:
            management.Append(sourceALRS+r"\KHUB.RH."+event, ALRSevent, "NO_TEST", '', '', 'Len(RouteID)<14')
            #locref.CreateLRSEventFromExistingDataset(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH.LRS_County", ALRSevent, "EventID", "RouteID", "LRSFromDate", "LRSToDate", "LocError", "Measure")
            eventcount = management.GetCount(ALRSevent)
            sourcecount = management.GetCount(sourceALRS+r"\KHUB.RH."+event)
            print(str(eventcount)+" of "+ str(sourcecount)+" Prod features Appended to model event for "+event)
        except:
            print("cant append prod data for "+event)
    else:
        #print('converting....')
        conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH."+event, proALRSdb+r"\\"+DBName+".RH.ALRS", event, '1=0', '',"NETWORK_GEOMETRY")
        #print ('altering...')
        management.AlterField(proALRSdb+r'\\'+DBName+'.RH.ALRS\\'+ event, "RouteID", "RouteID", "RouteID", "TEXT", "13", "NULLABLE")
        management.Append(sourceALRS+r"\KHUB.RH."+event, proALRSdb+r'\\'+DBName+'.RH.ALRS\\'+ event, "NO_TEST", '', '', 'Len(RouteID)<14')

        eventcount = management.GetCount(ALRSevent)
        sourcecount = management.GetCount(sourceALRS+r"\KHUB.RH."+event)
        print(str(eventcount)+" of "+ str(sourcecount)+" features from prod added to "+event + " not from model")
        #locref.CreateLRSEventFromExistingDataset(proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH.LRS_County", proALRSdb+r"\\"+DBName+".RH.ALRS\\"+DBName+".RH."+event, "EventID", "RouteID", "LRSFromDate", "LRSToDate", "LocError", "Measure")

ev_AADT Exists in ALRS Dataset with 23553 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_AADT2012 Exists in ALRS Dataset with 11313 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_AADT2013 Exists in ALRS Dataset with 10291 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_AADT2014 Exists in ALRS Dataset with 10431 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_AADT2015 Exists in ALRS Dataset with 10728 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_AADT2016 Exists in ALRS Dataset with 10771 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_AADTD

This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_MPOJurisdiction Exists in ALRS Dataset with 163 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_MuniROW Exists in ALRS Dataset with 179788 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_NG911RouteStatus Exists in ALRS Dataset with 119730 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_NG911StreetName Exists in ALRS Dataset with 270532 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_NHSFederalAid Exists in ALRS Dataset with 505 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_NonStateRouteStatus Exists in AL

ev_SurfaceWidthPaved Exists in ALRS Dataset with 16755 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_TerrainType Exists in ALRS Dataset with 5554 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_ThroughLaneWidth Exists in ALRS Dataset with 7809 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_TrafficSequence Exists in ALRS Dataset with 42309 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_TravelWayOperation Exists in ALRS Dataset with 126391 features 
This feature class is already registered with an LRS.
Failed to execute (CreateLRSEventFromExistingDataset).

ev_TruckRoute Exists in ALRS Dataset with 629 features 
This feature class is already registered with an LRS.
Failed to execute (Creat

In [ ]:
from arcpy import management
management.GetCount("KHub4.RH.ev_ReferencePost")